In [33]:
import os
import scipy.stats
from data import BBDataset
from torch.utils.data import DataLoader
from models.model import SAAN
import torch.nn as nn
import torch
import torch.optim as optim
from common import *
import argparse
import torch.nn.functional as F
import pandas as pd
import scipy
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [35]:
import sys
sys.path.append('/kaggle/input/baid-reproducibility/BAID-Reproducibility')


In [39]:
import os
import scipy.stats
from data import BBDataset
from torch.utils.data import DataLoader
from models.model import SAAN
import torch.nn as nn
import torch
import torch.optim as optim
from common import *
import argparse
import torch.nn.functional as F
import pandas as pd
import scipy
from tqdm import tqdm
test_dataset = BBDataset(file_dir='/kaggle/input/baid-reproducibility/BAID-Reproducibility/dataset', type='validation', test=True)

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--device', type=str, default='cuda')
    parser.add_argument('--checkpoint_dir', type=str,
                        default='/kaggle/input/baid-reproducibility/BAID-Reproducibility/checkpoint/BAID')
    parser.add_argument('--checkpoint_name', type=str,
                        default='/kaggle/input/baid-reproducibility/BAID-Reproducibility/model_best.pth')
    parser.add_argument('--save_dir', type=str,
                        default='/kaggle/input/baid-reproducibility/BAID-Reproducibility/result')
    return parser.parse_args()
def test(args):
    device = args.device
    checkpoint_path = os.path.join(args.checkpoint_dir, args.checkpoint_name)
    df = pd.read_csv('/kaggle/input/baid-reproducibility/BAID-Reproducibility/dataset/test_set.csv')
    predictions = []
    model = SAAN(num_classes=1)
    model = model.to(device)
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True, num_workers=8)
    with torch.no_grad():
        for step, test_data in tqdm(enumerate(test_loader)):
            image = test_data[0].to(device)
            predicted_label = model(image)
            prediction = predicted_label.squeeze().cpu().numpy()
            predictions.append(prediction * 10)
    scores = df['score'].values.tolist()
    print(scipy.stats.spearmanr(scores, predictions))
    print(scipy.stats.pearsonr(scores, predictions))
    acc = 0
    for i in range(len(scores)):
        cls1 = 1 if scores[i] > 5 else 0
        cls2 = 1 if predictions[i] > 5 else 0
        if cls1 == cls2:
            acc += 1
    print(acc/len(scores))
    df.insert(loc=2, column='prediction', value=predictions)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    save_path = os.path.join(args.save_dir, '/kaggle/input/baid-reproducibility/BAID-Reproducibility/result.csv')
    df.to_csv(save_path, index=False)

if __name__ == '__main__':

    args = parse_args()

    test(args)


100%|██████████| 3200/3200 [00:00<00:00, 228276.97it/s]
usage: ipykernel_launcher.py [-h] [--device DEVICE]
                             [--checkpoint_dir CHECKPOINT_DIR]
                             [--checkpoint_name CHECKPOINT_NAME]
                             [--save_dir SAVE_DIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-a76fd095-baf7-42aa-b2f5-6523786d29b9.json


SystemExit: 2

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
